In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 4 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

14 November 2022 ; 21:05:13


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
9,Ron Harper Jr.,SF,22,TOR,1,0,3.0,0.0,1.0,0.000,...,2.36,114.37,112.28,2.09,3.24,114.90,111.90,3.00,1,1
11,Christian Koloko,C,22,TOR,14,8,17.1,1.4,3.5,0.408,...,2.36,114.37,112.28,2.09,3.24,114.90,111.90,3.00,1,0
21,Kennedy Chandler,PG,20,MEM,6,0,9.7,1.0,3.2,0.316,...,1.36,114.69,113.30,1.40,1.17,114.04,112.81,1.23,1,1
25,Jake LaRavia,PF,21,MEM,11,0,18.7,1.9,4.3,0.447,...,1.36,114.69,113.30,1.40,1.17,114.04,112.81,1.23,1,1
26,Kenneth Lofton Jr.,PF,20,MEM,3,0,4.7,1.3,3.0,0.444,...,1.36,114.69,113.30,1.40,1.17,114.04,112.81,1.23,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.6,ORL,19.3,1.490909,5.963636,8.1,23.5,13,0.566602
1,Bennedict Mathurin,28.2,IND,17.2,1.366667,4.783333,6.1,19.9,6,0.403056
2,Jaden Ivey,31.4,DET,14.9,1.261538,1.892308,5.7,15.8,14,0.122002
3,Walker Kessler,14.7,UTA,20.7,1.366667,4.783333,2.3,5.4,3,0.053790
4,AJ Griffin,12.1,ATL,18.5,1.822222,3.644444,2.7,6.9,3,0.051097


In [8]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)